In [1]:
import nltk
import heapq
import numpy as np
nltk.download('stopwords')
nltk.download('punkt')
#from nltk.corpus import stopwords
import re

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
from google.colab import drive

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
import json

with open('/content/gdrive/My Drive/dataset_43428_1.txt', encoding="utf-8") as json_file:
    df = json.load(json_file)

In [4]:
print(df[0])

В Сербии арестован последний военный преступник
Задержан Горан Хаджич , разыскиваемый Гаагским трибуналом за военные преступления в ходе войны на Балканах .

Как сообщает « Лента.ру » , первые неофициальные сообщения об аресте Горана Хаджича появились ещё 19 июля , однако вскоре были опровергнуты .
В ближайшее время о поимке Хаджича должен официально объявить президент Сербии Борис Тадич .
Об этом сообщает сербский телеканал B92 .

Хорватский политик сербского происхождения Горан Хаджич был руководителем Республики Сербская Краина — непризнанного сербского государства , существовавшего на территории Хорватии в 1992 — 1995 годах .
В 2004 году Международный трибунал по бывшей Югославии в Гааге предъявил Хаджичу обвинения по 14 эпизодам военных преступлений и преступлений против человечности .
В частности , ему вменяется причастность к депортациям и убийствам мирного хорватского населения в 1991 — 1993 годах .

За информацию , способную помочь в поимке Горана Хаджича , в США была назначен

In [0]:
stopwords = nltk.corpus.stopwords.words('russian')

In [0]:
result = []
def claer_text(txt):
  txt = txt.strip()
  txt = txt.replace('\n', ' ')
  txt = txt.replace(' .', '.')
  txt = txt.replace(' ,', ',')
  txt = txt.replace(' ;', ';')
  txt = txt.replace(' :', ':')
  txt = txt.replace(' ?', '?')
  txt = txt.replace(' !', '!')
  txt = txt.replace(' )', ')')
  txt = txt.replace('( ', '(')
  txt = txt.replace('« ', '«')
  txt = txt.replace(' »', '»')
  txt = txt.replace('&quot;', '"')
  return txt
for texts in df:
  texts = claer_text(texts)
  texts = re.sub(r'\[[0-9]*\]', ' ', texts)
  texts = re.sub(r'\s+', ' ', texts)

  formatted_texts = re.sub('[^а-яА-Яa-zA-Z]', ' ', texts )
  formatted_texts = re.sub(r'\s+', ' ', formatted_texts)

  sentence_list = nltk.sent_tokenize(texts)

  word_frequencies = {}
  for word in nltk.word_tokenize(formatted_texts):
    if word not in stopwords:
      if word not in word_frequencies.keys():
        word_frequencies[word] = 1
      else:
        word_frequencies[word] += 1
  maximum_frequncy = max(word_frequencies.values())

  for word in word_frequencies.keys():
    word_frequencies[word] = (word_frequencies[word]/maximum_frequncy)

  sentence_scores = {}
  for sent in sentence_list:
    for word in nltk.word_tokenize(sent.lower()):
      if word in word_frequencies.keys():
        if sent not in sentence_scores.keys():
          sentence_scores[sent] = word_frequencies[word]
        else:
          sentence_scores[sent] += word_frequencies[word]
  summary_sentences = heapq.nlargest(len(sentence_list), sentence_scores, key=sentence_scores.get)
  len_sum_sentences = [len(i) for i in summary_sentences]
  x = 1
  summary = summary_sentences[0]
  while len(summary) < 300 and x < len(summary_sentences):
    if 300 - len(summary) > len_sum_sentences[x]/2:
      summary += ' ' + summary_sentences[x]
      x += 1
    else:
      break
  summary = summary.strip()
  result.append(summary)

In [0]:
with open('/content/gdrive/My Drive/outdataset_43428_1.txt', 'w', encoding='utf-8') as f:
    json.dump(result, f, ensure_ascii=False, indent=4)